In [110]:
import requests
import os
import json
from pprint import pprint

os.environ['BEARER_TOKEN'] = 'WRITE YOUR TOKEN HERE'
bearer_token = os.environ.get('BEARER_TOKEN')

def create_url(ids):
    tweet_fields = "tweet.fields=lang,author_id,attachments,context_annotations,conversation_id,created_at,entities,geo,id,public_metrics,withheld"
    expansion = "expansions=author_id"
    author_fields = "user.fields=id,name,username,created_at,description,entities,location,public_metrics,protected,verified,withheld"
    
    url = "https://api.twitter.com/2/tweets/{}?{}&{}&{}".format(ids, tweet_fields, expansion, author_fields)
    return url


def get_tweet(_id):
    url = create_url(_id)
    payload={}
    headers = {
      'Authorization': 'Bearer {}'.format(bearer_token),
      'Cookie': 'guest_id=v1%3A164999832374703747; guest_id_ads=v1%3A164999832374703747; guest_id_marketing=v1%3A164999832374703747; personalization_id="v1_+F68isE/iukb7yr8y66bOw=="'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    
    return response.json()

'''
def main():
    url = create_url()
    json_response = connect_to_endpoint(url)
    print(json.dumps(json_response, indent=4, sort_keys=True))
'''
'''
if __name__ == "__main__":
    main()
    '''

'\nif __name__ == "__main__":\n    main()\n    '

In [108]:
get_tweet(1287153210990395392)

ConnectionError: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/tweets/1287153210990395392?tweet.fields=lang,author_id,attachments,context_annotations,conversation_id,created_at,entities,geo,id,public_metrics,withheld&expansions=author_id&user.fields=id,name,username,created_at,description,entities,location,public_metrics,protected,verified,withheld (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001E239E54B88>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [35]:
import pandas as pd

def read_file(file_name):
    '''
    This function will read the file containing tweet ids sperating by lines (in groups)
    '''
    all_ids = pd.read_table(file_name,sep='\n',header=None)
    return all_ids

In [31]:
def separate_ids(ids):
    '''
    This function will split the sperate ids into a list
    '''
    seperated_ids = ids.split(",")
    return seperated_ids

In [60]:
def get_tweet_groups(groups_ids):
    '''
    This function will get tweets information using Twitter API in groups and indexing them
    params groups_ids: all ids in lines (groups) retrieved from the orginal file 
    '''
    tweet_groups = {}
    for group_index in range(len(groups_ids)):
        group_separeted_ids =  separate_ids(train_ids[0][group_index])

        group_dict = {}
        index_in_group = 0
        for _id in group_separeted_ids:
            group_dict[index_in_group] = get_tweet(_id)
            index_in_group += 1
        tweet_groups[group_index] = group_dict
    return tweet_groups

In [64]:
train_ids = read_file('../../project-data/train.data.txt')
tweets_train = get_tweet_groups(train_ids[0:2])

In [73]:
pprint(tweets_train[0])

{0: {'data': {'attachments': {'media_keys': ['3_1250219288406810624']},
              'author_id': '312362691',
              'context_annotations': [{'domain': {'description': 'Ongoing News '
                                                                 'Stories like '
                                                                 "'Brexit'",
                                                  'id': '123',
                                                  'name': 'Ongoing News Story'},
                                       'entity': {'id': '1220701888179359745',
                                                  'name': 'COVID-19'}}],
              'conversation_id': '1250217682533642240',
              'created_at': '2020-04-15T00:28:03.000Z',
              'entities': {'urls': [{'display_url': 'pic.twitter.com/ccMjhhD7BK',
                                     'end': 119,
                                     'expanded_url': 'https://twitter.com/ucoptempe/status/1250219300389974016

                                                                  'like '
                                                                  "'Brexit'",
                                                   'id': '123',
                                                   'name': 'Ongoing News '
                                                           'Story'},
                                        'entity': {'id': '1220701888179359745',
                                                   'name': 'COVID-19'}},
                                       {'domain': {'description': 'Entity '
                                                                  'Service '
                                                                  'related '
                                                                  'Events '
                                                                  'domain',
                                                   'id': '29',
                                          